In [1]:
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [2]:
tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        OneHotDegree(max_degree=8,cat=False)
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)


print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 9


C:\Users\ernst\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch_geometric\data\storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'betti_numbers', 'dimension', 'n_vertices', 'name', 'torsion_coefficients', 'face', 'genus', 'orientable'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Number of classes: 2

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 9], y=[1])
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


In [3]:
data.betti_numbers

[1, 0, 1]

In [4]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")


Number of training graphs: 562
Number of test graphs: 150


In [5]:
train_loader = DataLoader(train_dataset,batch_size=10)#,sampler=ImbalancedSampler(train_dataset))
test_loader = DataLoader(test_dataset,batch_size=10)


for batch in train_loader:
    break

batch.y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
from operator import concat
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TAGConv,TransformerConv
from torch_geometric.nn import global_mean_pool
from torch_scatter import segment_coo
import torch.nn as nn



class PermInvariant(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # torch.manual_seed(12345)
        self.classification = nn.Sequential( 
            nn.Linear(dataset.num_node_features,hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels,hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels,3)
        )

    def forward(self, batch):
        x = self.classification(batch.x)
        # print(batch.x)
        # print(x)
        return segment_coo(x,batch.batch,reduce="sum")


model = PermInvariant(hidden_channels=64)
print(model(batch))


tensor([[-1.2394, -0.3322,  0.7674],
        [-1.1591, -0.6497,  0.7996],
        [-1.0816, -0.3613,  0.6788],
        [-1.1017, -0.5386,  0.8528],
        [-1.2463, -0.3314,  0.7317],
        [-1.2013, -0.5507,  0.7876],
        [-1.0896, -0.5982,  0.8156],
        [-1.1058, -0.6580,  0.8609],
        [-1.1820, -0.3744,  0.8095],
        [-1.2314, -0.4554,  0.7725]], grad_fn=<CppNode<class SegmentSumCOO>>)


In [7]:
model = PermInvariant(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data)  # Perform a single forward pass.
        loss = criterion(out, torch.tensor(data.betti_numbers,dtype=torch.float))  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    losses = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)
        
        losses += criterion(out, torch.tensor(data.betti_numbers,dtype=torch.float))
    return losses


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 6.9451, Test Acc: 1.6976
Epoch: 002, Train Acc: 6.9317, Test Acc: 1.6716
Epoch: 003, Train Acc: 6.9236, Test Acc: 1.6698
Epoch: 004, Train Acc: 6.9116, Test Acc: 1.6674
Epoch: 005, Train Acc: 6.9051, Test Acc: 1.6663
Epoch: 006, Train Acc: 6.9046, Test Acc: 1.6669
Epoch: 007, Train Acc: 6.9090, Test Acc: 1.6681
Epoch: 008, Train Acc: 6.9065, Test Acc: 1.6686
Epoch: 009, Train Acc: 6.9074, Test Acc: 1.6681
Epoch: 010, Train Acc: 6.9103, Test Acc: 1.6705
Epoch: 011, Train Acc: 6.8380, Test Acc: 1.6461
Epoch: 012, Train Acc: 6.9187, Test Acc: 1.6731
Epoch: 013, Train Acc: 6.9225, Test Acc: 1.6763
Epoch: 014, Train Acc: 6.9648, Test Acc: 1.6940
Epoch: 015, Train Acc: 6.9375, Test Acc: 1.6826
Epoch: 016, Train Acc: 6.9334, Test Acc: 1.6820
Epoch: 017, Train Acc: 6.9338, Test Acc: 1.6821
Epoch: 018, Train Acc: 6.9380, Test Acc: 1.6837
Epoch: 019, Train Acc: 6.9421, Test Acc: 1.6853
Epoch: 020, Train Acc: 6.9448, Test Acc: 1.6870
Epoch: 021, Train Acc: 6.9383, Test Acc:

In [8]:
model.eval()
losses = 0
y_hat = []
y = []
for data in test_loader:  # Iterate in batches over the training/test dataset.
    out = model(data)
    y_hat.append(out.round().long())
    y.append(torch.tensor(data.betti_numbers))

y_hat = torch.vstack(y_hat)
y = torch.vstack(y)


incorrect = torch.count_nonzero(y-y_hat!=0).item()
correct = torch.count_nonzero(y-y_hat==0).item()

print("predicted incorrect",correct)
print("predicted incorrect",incorrect)
print("percentage correct", correct / (correct + incorrect))


predicted incorrect 371
predicted incorrect 79
percentage correct 0.8244444444444444


In [12]:
y

tensor([[1, 2, 0],
        [1, 2, 1],
        [1, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 2, 0],
        [1, 1, 0],
        [1, 1, 0],
        [1, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 2, 0],
        [1, 1, 0],
        [1, 2, 0],
        [1, 0, 1],
        [1, 0, 0],
        [1, 2, 1],
        [1, 1, 0],
        [1, 0, 0],
        [1, 0, 1],
        [1, 2, 1],
        [1, 2, 1],
        [1, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [1, 2, 1],
        [1, 2, 0],
        [1, 0, 0],
        [1, 1, 0],
        [1, 0, 0],
        [1, 3, 0],
        [1, 1, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 1],
        [1, 2, 1],
        [1, 2, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 1],
        [1, 2, 1],
        [1, 2, 0],
        [1, 2, 1],
        [1, 1, 0],
        [1, 2, 1],
        [1, 1, 0],
        [1, 2, 0],
        [1, 0, 1],
        [1, 0, 0],
        [1, 1, 0],
        [1, 